In [18]:
!pip install --user cython

    100% |████████████████████████████████| 3.3MB 236kB/s ta 0:00:01
You are using pip version 9.0.3, however version 10.0.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [19]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
# from scipy.sparse import csr_matrix, block_diag
from tqdm import tqdm

In [20]:
# import xgboost as xgb

In [21]:
from joblib import Parallel, delayed

In [22]:
import pickle

In [23]:
from line_profiler import LineProfiler
%load_ext Cython

In [24]:
def profile_print(func_to_call, *args):
    profiler = LineProfiler()
    profiler.add_function(func_to_call)
    profiler.runcall(func_to_call, *args)
    profiler.print_stats()

In [27]:
train = pd.read_csv("data/prepared_train.csv")#, nrows=100)
# logging.info("data was read.")
# inds = []
# for qid in sorted(set(train.QID), reverse=True):
#     prev_y = -1
#     for y_i in train[train.QID == qid].index:
#         if train.Y[y_i] != prev_y:
#             prev_y = train.Y[y_i]
#             inds.append(y_i)
# # train = train.loc[inds[:2]]

# train = train.reset_index(drop=True)

X_columns = [col for col in train.columns if col[0] == u'X']
train_colums = ['Y', 'QID'] + X_columns

In [28]:
data = train

In [8]:
# data.sort_values(['QID', 'Y'], ascending=True, kind='mergesort').to_csv('./train.data.sorted.cvs', index=None)

In [165]:
st = 0
end = 30000
data = data.loc[st:end,:]
data.index = [i for i in range(0, end-st+1)]

In [9]:
X_train = data.iloc[:, 1:-1]
y_train = data['Y']

In [10]:
bounds = []
st = 0
end = 1

for i in range(1, X_train.shape[0]):
    if data['QID'][end] == data['QID'][st]:
        end += 1
    else:
        bounds.append((st, end))
        st = end
        end += 1
bounds.append((st, end))
bounds = np.array(bounds, dtype=int)

In [11]:
# %%cython -a

# import numpy as np
# cimport numpy as np

# def func(np.ndarray[np.float64_t, ndim=1] arr, np.ndarray[long, ndim=2] b):
#     cdef np.float64_t rk, prev
#     cdef long i, j, cnt
#     cpdef np.ndarray[np.float64_t, ndim=1] result 
    
#     result = np.zeros(arr.shape[0])
#     cnt = 0
#     for i in range(b.shape[0]):
#         cnt += 1
#         rk = 4
#         prev = arr[b[i, 1] - 1]
        
#         for j in range(b[i, 1] - 1, b[i, 0] - 1, -1):
#             if arr[j] != prev:
#                 rk -= 1
            
#             prev = arr[j]
#             result[j] = rk

#         if cnt % 500 == 0:
#             print(cnt)    
#     return result

In [12]:
# result = func(np.array(data['Y'], dtype=float), np.array(bounds, dtype=int))

In [13]:
# data['Y'] = result
# data.to_csv('./train.data.sorted-rank.cvs', index=None)

In [14]:
%%cython -a

import numpy as np
cimport numpy as np
from libc.math cimport exp, log2, fabs

cdef np.float64_t sigm(np.float64_t x):
    if x < -20.0:
        return 0.0
    if x > 20.0:
        return 1.0
    
    return 1.0 / (1.0 + exp(-x))

def gradient(np.ndarray[np.float64_t, ndim=1] preds, np.ndarray[np.float64_t, ndim=1] y, np.ndarray[long, ndim=2] b):
    cpdef np.ndarray[np.float64_t, ndim=1] result = np.zeros(y.shape[0]), hess = np.zeros(y.shape[0])
    cdef long i, j, k, posi, posj, l
    cdef np.float64_t r, s, gr, delta, reli, relj, mmax, ndcg
    cpdef np.ndarray[long, ndim=1] ranks, buf
    
    for k in range(len(b)):
        l = b[k, 1] - b[k, 0]
        
        buf = np.argsort(preds[b[k, 0] : b[k, 1]])
        ranks = np.zeros(buf.shape[0], dtype=long)
        for j in range(buf.shape[0]):
            ranks[buf[j]] = j
        
        ndcg = 0.0
        for i in range(b[k, 0], b[k, 1]):
            posi = int(y[i])
            reli = (1 << posi) - 1.0
            ndcg += reli / (log2(b[k, 1] - i) + 1.0)

        for i in range(b[k, 0], b[k, 1]):
            for j in range(b[k, 0], i):
                if y[i] == y[j]:
                    continue
                    
                r = preds[i] - preds[j]

                posi = int(y[i])
                posj = int(y[j])
                reli = ((1 << posi) - 1.0)
                relj = ((1 << posj) - 1.0)
    
                delta = relj / (log2(l - ranks[i - b[k, 0]]) + 1.0) 
                delta += reli / (log2(l - ranks[j - b[k, 0]]) + 1.0)
                delta -= reli / (log2(l - ranks[i - b[k, 0]]) + 1.0)
                delta -= relj / (log2(l - ranks[j - b[k, 0]]) + 1.0)
                delta = fabs(delta / ndcg)
                
                gr = -sigm(-r) * delta
                result[i] += gr
                result[j] -= gr
                hess[i] += -gr * sigm(r)
                hess[j] += -gr * sigm(r)
        
    return result, hess

In [26]:
def parallel_gradient(preds, y, n_jobs):
    per_job = int(np.ceil(len(bounds) * 1.0 / n_jobs))
    results = Parallel(n_jobs=n_jobs)(delayed(gradient)(preds, y, bounds[i * per_job : (i+1) * per_job]) for i in range(n_jobs))

    result = np.zeros(y.shape[0])
    hess = np.zeros(y.shape[0])
    
    for arr1, arr2 in results:
        result += arr1
        hess += arr2
    return result, hess

In [18]:
# def loss(preds, y, b):
#     result = 0.0
#     cnt = 0
    
#     for p in b:
#         for i in range(p[0], p[1]):
#             for j in range(p[0], i):
#                 s = np.sign(y[i] - y[j])
#                 r = preds[i] - preds[j]
# #                 cnt += 1
#                 result += 0.5 * (1 - s) * sigm(r) + np.log(1 + np.exp(-sigm(r)))
#     return result

In [19]:
%%cython -a

import numpy as np
cimport numpy as np
from libc.math cimport exp, log, log2, fabs

cdef np.float64_t sigm(np.float64_t x):
    if x < -20.0:
        return 0.0
    if x > 20.0:
        return 1.0
    
    return 1.0 / (1.0 + exp(-x))

cdef np.float64_t sigm1(np.float64_t x):
    return sigm(x) * (1 - sigm(x))

def loss(np.ndarray[np.float64_t, ndim=1] preds, np.ndarray[np.float64_t, ndim=1] y, np.ndarray[long, ndim=2] b):
    cdef np.float64_t result, r, s, ndcg, ranki, rankj
    cdef long i, j, k, posi, posj, l
    cpdef np.ndarray[long, ndim=1] ranks, buf
    
    result = 0.0
    
    for k in range(len(b)):
        l = b[k, 1] - b[k, 0]
        
        buf = np.argsort(preds[b[k, 0] : b[k, 1]])
        ranks = np.zeros(buf.shape[0], dtype=long)
        for j in range(buf.shape[0]):
            ranks[buf[j]] = j
        
        ndcg = 0.0
        for i in range(b[k, 0], b[k, 1]):
            posi = int(y[i])
            reli = (1 << posi) - 1.0
            ndcg += reli / (log2(b[k, 1] - i) + 1.0)
            
        for i in range(b[k, 0], b[k, 1]):
            for j in range(b[k, 0], i):
                if y[i] == y[j]:
                    continue

                r = preds[i] - preds[j]
                
                posi = int(y[i])
                posj = int(y[j])
                reli = ((1 << posi) - 1.0)
                relj = ((1 << posj) - 1.0)
    
                delta = relj / (log2(l - ranks[i - b[k, 0]]) + 1.0) 
                delta += reli / (log2(l - ranks[j - b[k, 0]]) + 1.0)
                delta -= reli / (log2(l - ranks[i - b[k, 0]]) + 1.0)
                delta -= relj / (log2(l - ranks[j - b[k, 0]]) + 1.0)
                delta = fabs(delta / ndcg)
                
                if r < -20.0:
                    result += (-r) * delta
                elif r > 20:
                    result += 0.0
                else:
                    result += log(1.0 + exp(-r)) * delta
    return result

In [21]:
def parallel_loss(preds, y, n_jobs):
    per_job = int(np.ceil(len(bounds) * 1.0 / n_jobs))
    results = Parallel(n_jobs=n_jobs)(delayed(loss)(preds, y, bounds[i * per_job : (i+1) * per_job]) for i in range(n_jobs))

    return sum(results)

In [22]:
%%cython -a

import numpy as np
cimport numpy as np

from libc.math cimport log2

def ndcg_loss(np.ndarray[np.float64_t, ndim=1] preds, np.ndarray[np.float64_t, ndim=1] y, np.ndarray[long, ndim=2] b):
    cdef long i, j, l, posi
    cdef np.float64_t result, ndcg, reli, tmp
    cpdef np.ndarray[long, ndim=1] ranks, buf
    
    result = 0.0
    
    for k in range(len(b)):
        ndcg = 0.0
        tmp = 0.0
        
        buf = np.argsort(preds[b[k, 0] : b[k, 1]])
        ranks = np.zeros(buf.shape[0], dtype=long)
        for j in range(buf.shape[0]):
            ranks[buf[j]] = j
        
        l = b[k, 1] - b[k, 0]
        
#         print(ranks)
        
        for i in range(b[k, 0], b[k, 1]):
            posi = int(y[i])
            reli = (1 << posi) - 1.0
            ndcg += reli / (log2(b[k, 1] - i) + 1.0)
            tmp += reli / (log2(l - ranks[i - b[k, 0]]) + 1.0)
        
#             print(i, reli, (log2(l - ranks[i - b[k, 0]]) + 1.0), )
        
        result += tmp / ndcg
        
    return result / len(b)

In [28]:
class GDB:
    def __init__(self, n_epoch=10, max_depth=3, verbose=0, n_jobs=2, lr=1.0):
        self.n_epoch = n_epoch
        self.max_depth = max_depth
        self.verbose = verbose
        self.n_jobs = n_jobs
        self.loaded = False
        self.lr = lr
        self.estimators_ = []
        self.m = 0
        self.h = []
        
    def fit(self, X, y):
        if not self.loaded:
            self.h = np.zeros(X.shape[0])
            self.m = 0.0
        
#         print('Loss', parallel_loss(self.h, y, self.n_jobs))
        
        for i in range(self.n_epoch):
            g, hess = parallel_gradient(self.h, y, self.n_jobs)
            g *= -1
#             g, numin, denom = parallel_gradient(self.h, y, self.n_jobs)
            
#             print('-grad', g[50:60])
            
            sampled = X.sample(int(X.shape[0] * 0.5))
    
            tree = DecisionTreeRegressor(max_depth=self.max_depth)
            tree.fit(sampled, g[sampled.index])
            
#             print('numin', numin)
#             print('denom', denom)
            
            leaf_ids = tree.apply(X)
# #             print('leafs', leaf_ids)
            
            for idx in np.unique(leaf_ids):
                tree.tree_.value[idx, 0, 0] = g[leaf_ids == idx].sum() / hess[leaf_ids == idx].sum()
#                 print('n/d =', numin[leaf_ids == idx].sum() / denom[leaf_ids == idx].sum())
    
            self.estimators_.append(tree)
            self.h += self.lr * tree.predict(X)
            
#             if i % 10 == 0:
            if True:
                print('Epoch:', i, 'Loss:', parallel_loss(self.h, y, self.n_jobs))
                print('Total NDCG:', ndcg_loss(self.h, y, bounds))
                print('NDCG:', ndcg_loss(self.h, y, bounds[0:1]))
#                 print('h', tree.predict(X)[50:60])
                
                print('\n')
    
#                 if i != 0:
#                     self.save('./gdb' + '-e' + str(i) + '.bin')
    
    def predict(self, X):
        res = np.ones(X.shape[0]) * self.m
        
        for tree in self.estimators_:
            res += self.lr * tree.predict(X)
            
        return res
    
    def save(self, f_name):
        d = {'estimators_': self.estimators_, 'm': self.m, 'h': self.h}
        with open(f_name, 'wb') as fout:
            pickle.dump(d, fout)
            
    def load(self, f_name):
        d = {}
        with open(f_name, 'rb') as fin:
            d = pickle.load(fin)
            
        self.loaded = True
        self.estimators_ = d['estimators_']
        self.h = d['h']
        self.m = d['m']

In [29]:
gdb = GDB(n_epoch=3, max_depth=3, n_jobs=2, lr=1.0)
profile_print(gdb.fit, *(X_train, np.array(y_train)))
# gdb.fit(X_train, np.array(y_train))

Epoch: 0 Loss: 51880.635085284404
Total NDCG: 0.7731839689700272
NDCG: 0.7839954656249202


Epoch: 1 Loss: 47221.22718097067
Total NDCG: 0.7876505212518253
NDCG: 0.7737658125512652


Epoch: 2 Loss: 45996.2029184721
Total NDCG: 0.7903746103024917
NDCG: 0.7803389221520136


Timer unit: 1e-06 s

Total time: 38.2928 s
File: <ipython-input-28-f4d35abdcd89>
Function: fit at line 13

Line #      Hits         Time  Per Hit   % Time  Line Contents
    13                                               def fit(self, X, y):
    14         1            2      2.0      0.0          if not self.loaded:
    15         1         1249   1249.0      0.0              self.h = np.zeros(X.shape[0])
    16         1            2      2.0      0.0              self.m = 0.0
    17                                                   
    18                                           #         print('Loss', parallel_loss(self.h, y, self.n_jobs))
    19                                                   
    20       

In [ ]:
gdb = GDB(n_epoch=301, max_depth=4, n_jobs=4, lr=0.1)
# gdb.load('gdb.bin')
gdb.fit(X_train, np.array(y_train))

Epoch: 0 Loss: 63871.805523282426
Total NDCG: 0.7786338394473414
NDCG: 0.7649772255787416


Epoch: 10 Loss: 54688.14465120931
Total NDCG: 0.7937894248578776
NDCG: 0.8251910855846061


Epoch: 20 Loss: 51156.93090711422
Total NDCG: 0.7964947030194899
NDCG: 0.8612897431140066


Epoch: 30 Loss: 49423.76210817184
Total NDCG: 0.7967566554038692
NDCG: 0.8666642875099264


Epoch: 40 Loss: 48330.290563808
Total NDCG: 0.7993311356344474
NDCG: 0.8638978850409507


Epoch: 50 Loss: 47502.40657507908
Total NDCG: 0.8010642878346771
NDCG: 0.869927619576604


Epoch: 60 Loss: 46974.683785866175
Total NDCG: 0.8013635424418204
NDCG: 0.870380089149472


Epoch: 70 Loss: 46638.30087597123
Total NDCG: 0.8024497878096992
NDCG: 0.8703913504905224


Epoch: 80 Loss: 46312.61948703289
Total NDCG: 0.8031421072306989
NDCG: 0.8702550334618254


Epoch: 90 Loss: 46040.58521636054
Total NDCG: 0.8041967631365361
NDCG: 0.868459036362017


Epoch: 100 Loss: 45877.56759866213
Total NDCG: 0.8049939995309172
NDCG: 0.8610394811

In [226]:
loss(np.array(y_train) * 10, np.array(y_train), bounds)

1060.26869351626

In [176]:
loss(np.array(y_train), np.array(y_train), bounds)

285549.70170075126

In [237]:
ndcg_loss(gdb.h, np.array(y_train), bounds)

0.805942496879264

In [244]:
loss(gdb.h, np.array(y_train), bounds)

23964929.720876858

In [117]:
# gdb = GDB(n_epoch=100, max_depth=3, n_jobs=1)
# gdb.load('./gdb.bin')

In [361]:
parallel_loss(gdb.predict(X_train), np.array(y_train), 1)

813066.1018251859

In [28]:
y_train

0         1.09861
1         1.09861
2         1.09861
3         1.09861
4         1.09861
5         1.09861
6         1.09861
7     10987.20000
8     10987.20000
9     10987.20000
10    10987.20000
Name: Y, dtype: float64

In [29]:
gdb.predict(X_train)

array([  4.18444336e+066,  -5.80962280e+001,  -5.80962280e+001,
         1.73001151e+171,   8.38089029e+066,  -6.04433749e+001,
        -6.04433749e+001,  -6.04433749e+001,  -1.01517705e+004,
        -1.01519062e+004,              -inf])

In [81]:
test_data = pd.read_csv('testset.cvs')

In [82]:
X_test = test_data.iloc[:, 1:-1]

In [60]:
test_data.loc[50:140, :].head()

,Y,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X128,X129,X130,X131,X132,X133,X134,X135,X136,QID
50,-1,2,2,2,1,2,1.0,1.0,1.0,0.5,...,10,0,2340,3401,6,7,0,0,0.000000,13
51,-1,2,0,1,1,2,1.0,0.0,0.5,0.5,...,0,0,56509,26689,9,12,0,52,37.722367,13
52,-1,2,1,2,0,2,1.0,0.5,1.0,0.0,...,38,0,24693,6372,20,35,0,1,0.800000,13
53,-1,2,0,2,1,2,1.0,0.0,1.0,0.5,...,0,0,23359,3468,58,110,0,0,0.000000,13
54,-1,2,2,2,0,2,1.0,1.0,1.0,0.0,...,26532,10,62872,44156,1,3,0,31,60.645952,13


In [67]:
# res = 1.0
# for tree in gdb.estimators_:
#     res += 1.0 * tree.predict(X_test)

In [270]:
res = gdb.predict(X_test)

In [271]:
answer = []
qids = []
for idx in np.unique(test_data['QID']):
    shift = (test_data.loc[:, 'QID'] == idx).idxmax()
    idxs = np.argsort(res[test_data.loc[:, 'QID'] == idx])[-5:][::-1]
#     idxs = np.argsort(res[test_data.loc[:, 'QID'] == idx])[:10]
    idxs += shift + 1
    answer.extend(idxs)
    qids.extend(np.ones(idxs.shape[0], dtype=int) * idx)

In [272]:
total = pd.DataFrame(columns=['DocumentId','QueryId'])
total['DocumentId'] = answer
total['QueryId'] = qids

In [273]:
total.to_csv('./result.csv', index=None)